# Zadanie 4 - SmallVector:

## 4.Mamy klasę **SmallVector**, która działa podobnie jak [<boost/container/small_vector.hpp>](https://www.boost.org/doc/libs/1_79_0/doc/html/boost/container/small_vector.html), czyli zawiera 10 elementów w tablicy statycznej, a w razie przekroczenia elementów **wszystkie dane** znajdują się w tablicy dynamicznej (wtedy statyczna tablica jest nieużywana)

## grupa 1:

In [ ]:
class SmallVector {
    using T = int;
    static constexpr unsigned N_ = 10;
    unsigned size_ = 0, capacity_ = N_;
    T data1_[N_] {};
    T* data2_ = nullptr;
public:
    SmallVector() = default;
    void push_back(const T& newElement); // nie musimy tego implementować, ale możemy użyć
    // ... nasza implementacja
};

### a.Proszę do niej napisać: operator przypisania **przenoszący** [5p]:

In [ ]:
SmallVector& SmallVector::operator=(SmallVector &&old)
{
    if(this != &old)
    {
        size_ = old.size_;
        capacity_ = old.capacity_;
        old.size_ = old.capacity_ = 0;
        if(old.data2_)
        {
            data2_ = old.data2_;
            old.data2_ = nullptr;
        }
        else
        {
            std::copy(old.data1_, old.data1_+size_, data1_);
            // old.data1_ - nie trzeba tego zerować
        }
    }
    return *this;
}

alternatywna wersja, która używa [std::exchange](https://en.cppreference.com/w/cpp/utility/exchange):

In [ ]:
SmallVector& SmallVector::operator=(SmallVector &&old)
{
    if(this != &old)
    {
        size_ = std::exchange(old.size_, 0);
        capacity_ = std::exchange(old.capacity_, 0);

        if(size_ > N)
        {
            data2_ = std::exchange(old.data2_, nullptr);
        }
        else
        {
            std::copy(old.data1_, old.data1_+size_, data1_);
            // old.data1_ - nie trzeba tego zerować
        }
    }
    return *this;
}

### b.Proszę do powyższej klasy napisać konstruktor **kopiujący** [3p]:

In [ ]:
SmallVector::SmallVector(const SmallVector& other)
{
    if (other.size_ < N_)
    {
        for(decltype(size_) i = 0; i < other.size_; i++)
            data1_[i] = other.data1_[i];
        data2_ = nullptr;
    }
    else
    {
        data2_ = new int[other.size_];
        for(decltype(size_) i = 0; i < other.size_; i++)
            data2_[i] = other.data2_[i];
    }
    capacity_ = other.capacity_;
    size_ = other.size_;
}

II wersja, która oddelegowywuje kamsymalnie dużo na listę inicjalizacyjną, oraz używa [std::exchange](https://en.cppreference.com/w/cpp/utility/exchange):

In [ ]:
SmallVector::SmallVector(const SmallVector& other): 
    size_(other.size_),
    capacity_(other.capacity_),
    data2_(size_ > N ? new T[size_] : nullptr)
{
    std::copy(other.data1_, other.data1_ + other.size_, data1_);
    /// alternatywnie, gdyz dla statycznych tablic dziala:
    //std::copy(begin(other.data1_), end(other.data1_), begin(data1_);
}

### c.Proszę do niej napisać aby działała pętla zakresowe-for (chodzi o iteratory. Informacja co powinien zawierać iterator na ostatniej stronie) [4p]:

In [ ]:
T* SmallVector::begin()
{
    if(size_ < N_)
    { 
        return data1_; 
    }
    return data2_;
}

T* SmallVector::end()
{
    if(size_ < N_)
    { 
        return data1_ + size_; 
    }
    return data2_ + size_;
} 

// jak widać wystarczy zwrócić wskaźnik, nie trzeba definiować klasy `iterator`, gdyż mamy pamięć ciągłą!

II wersja, która używa operatora 3-argumentowego:

In [ ]:
T* SmallVector::begin()
{
    return (size_ < N_) ? data1_ : data2_;
}

T* SmallVector::end()
{
    return (size_ < N_) ? data1_ + size_ : data2_ + size_;
}

### częste błędy w tej grupie:

In [ ]:
/*
data1_ = other.data1_; - tak nie można bo to statyczna tablica!
data2_ = other.data2_; - przy kopiowaniu to spowoduje, że będą dwa wskaźniki wskazywać na tę samą pamięć
data2_.push_back(...); - to nie kontener i nie ta grupa:D
+ wiele z tych co dla grupy2
*/

## grupa 2:

In [ ]:
#include <array>
#include <vector>
#include <utility> // std::exchange()

class SmallVector {
    using T = int;
    static constexpr unsigned N_ = 10;
    unsigned size_ = 0;
    std::array<T, N_> data1_;
    std::vector<T> data2_;
public:
    SmallVector() = default;
    void push_back(const T& newElement); // nie musimy tego implementować, ale możemy użyć
    // ... nasza implementacja
};

### a.Proszę do niej napisać: operator przypisania **kopiujący** [5p]:

In [ ]:
SmallVector& SmallVector::operator=(const SmallVector& other)
{
    if(&other != this) {
        // tutaj mamy kontenery standardowe, więc one mają operator=
        data1_ = other.data1_;
        data2_ = other.data2_;
        size_ = other.size_;
    }
    return *this;
}

alternatywnie, można bazować na domyślnie wygenerowanym, czyli napisać jak poniżej (nienapisanie w ogóle nie przekonuje mnie, że ktoś wiedział, że się sam taki wygeneruje):

In [ ]:
SmallVector& SmallVector::operator=(const SmallVector& other) = default;

### b.Proszę do powyższej klasy napisać konstruktor **przenoszący** [3p]:

In [ ]:
SmallVector::SmallVector(SmallVector &&other) {
    data1_ = other.data1_;
    data2_ = other.data2_;
    size_ = other.size_;

    other.size_ = 0;
    other.data2_.clear();
}

II wersja, przez użycie `std::swap()` (implementacja poniżej)

In [ ]:
SmallVector::SmallVector(SmallVector &&other){
    std::swap(data1_, other.data1_);
    std::swap(data2_, other.data2_);
    std::swap(size_, other.size_);
}
/** Słowem wyjaśnienia - w klasie inicjalizujemy pole size_ = 0, dlatego taka wymiana w pełni załatwia sprawę **/

III wersja - użycie listy inicjalizacyjnej, aby zamiast wołać dla każdej składowej konstruktor domyślny i opierator= zawołać od razu odpowiedni konstruktor:

In [ ]:
SmallVector::SmallVector(SmallVector &&other) :
    size_(other.size_), data1_(other.data1_), data2_(other.data2_) {

    other.size_ = 0;
    other.data2_.clear();
}

IV wersja: ale na dobrą sprawę po co wołać wpierw konstruktory kopiujące aby potem czyścić zawartość kontenerów, jeśli my chcemy przenieść - od razu wołajmy konstruktory na liście inicjalizacyjnej:

In [ ]:
SmallVector::SmallVector(SmallVector &&other) :
    size_(std::exchange(other.size_, 0), data1_(std::move(other.data1_)), data2_(std::move(other.data2_)) 
{}

### c.Proszę do niej napisać aby działała pętla zakresowe-for (chodzi o iteratory. Informacja co powinien zawierać iterator na ostatniej stronie) [4p]:

In [ ]:
T* SmallVector::begin()
{
    return size_ > N_ ?
        &data2_[0] : &data1_[0];
}

T* SmallVector::end()
{
    return size_ > N_ ?
        &data2_[size_] : &data1_[size_];
}

### częste błędy w tej grupie:

In [ ]:
// przepisywanie od sąsiada z innej grupy!
// czytanie ze zrozumieniem (operator= zamiast konstruktora, przenoszenie zamiast kopiowania)
//        delete data1_; - to jest kontener, więc nie usuwamy
//        delete[] data2_; - jw.
//        data2_.resize(other.data2_.size());
//        std::copy(other.data2_.begin(), other.data2_.end(), data2_.begin()); - zanim tak zrobimy musi być miejsce, czyli musimy zawołać: data2_.resize(other.data2_.size());
//          alternatywnie można użyć std::back_inserter (wtedy nie trzeba wołać resize()):
//         std::copy(other.data2_.begin(), other.data2_.end(), back_inserter(data2_));